In [1]:
import numpy as np
from srp import srp
p = 500 # Number of dimensions
n0 = 150 # Number of in-control samples
n1 = 150 # Number of out-of-control samples
Tmax = n0+n1
delta = 1 # Change magnitude
c = 1 # Target meanshift is c * M
k = 20 # Number of failuer Mode
sel_failure_mode = [0,1,2,3,4,5]
M = np.kron(np.eye(k),np.ones((int(p/k),1))) # Mean Failure Modes
Ks = 5

nsensors = 5

In [2]:
def data_gen_func0(n_batch, Tmax, seed):
    # Generate normal samples
    np.random.seed(seed)    
    data = np.random.randn(n_batch,Tmax,p)
    return data

def data_gen_func1(n_batch, Tmax, seed, T0, delta):
    # Generate abnormal samples
    np.random.seed(seed)
    data = np.random.randn(n_batch, Tmax, p)
    data[:,T0:,:] = data[:,T0:,:] + np.sum(M[:,sel_failure_mode]*delta,1)
    return data


### Use Extended TSSRP 

In [10]:
from ExtendedTSSRP import ExtendedTSSRP
from spc import spc

k = 20 # Number of failuer Mode
sel_failure_mode = [0,1,2,3,4,5]
M = np.kron(np.eye(k),np.ones((int(p/k),1))) # Mean Failure Modes

ETSSRP_srp_model = ExtendedTSSRP(p, c, k,M,nsensors,Ks)
ETSSRP_monitor_statistics = lambda x, T0, L: ETSSRP_srp_model.compute_monitor_batch(x,T0, L)
ETSSRP_spc_model = spc(ETSSRP_monitor_statistics,data_gen_func0, data_gen_func1)

n_batch = 2
Tmax = 500
seed_list = np.arange(10)
L = ETSSRP_spc_model.phase1(n_batch, Tmax, seed_list,r=1000,ARL0=100)
n_batch = 20


In [11]:
T_all, RL = ETSSRP_spc_model.phase2( n_batch, Tmax, np.arange(100,110), 20, delta=1)
np.mean(RL)

3.635

### Use TSSRP itself


In [12]:
from TSSRP import TSSRP
from spc import spc

k = p
M = np.eye(p)
vanilla_TSSRP_srp_model = TSSRP(p, c, k,M,nsensors,Ks)
vanilla_TSSRP_monitor_statistics = lambda x, T0, L: vanilla_TSSRP_srp_model.compute_monitor_batch(x,T0, L)
vanilla_TSSRP_spc_model = spc(vanilla_TSSRP_monitor_statistics,data_gen_func0, data_gen_func1)

n_batch = 2
Tmax = 500
seed_list = np.arange(10)
L = vanilla_TSSRP_spc_model.phase1(n_batch, Tmax, seed_list,r=1000,ARL0=100)
n_batch = 20


In [13]:
T_all, RL = vanilla_TSSRP_spc_model.phase2( n_batch, Tmax, np.arange(100,110), 20, delta=1)
np.mean(RL)

23.255

### Use Extended TSSRP to simulate TSSRP by setting M = I

In [10]:
from ExtendedTSSRP import ExtendedTSSRP
from spc import spc

M = np.eye(p)
k = p
TSSRP_srp_model = ExtendedTSSRP(p, c, k,M,nsensors,Ks)
TSSRP_monitor_statistics = lambda x, T0, L: TSSRP_srp_model.compute_monitor_batch(x,T0, L)
TSSRP_spc_model = spc(TSSRP_monitor_statistics,data_gen_func0, data_gen_func1)

n_batch = 2
Tmax = 500
seed_list = np.arange(10)
L = TSSRP_spc_model.phase1(n_batch, Tmax, seed_list,r=1000,ARL0=100)
n_batch = 20


In [14]:
T_all, RL = TSSRP_spc_model.phase2( n_batch, Tmax, np.arange(100,110), 20, delta=0.5)
np.mean(RL)

33.74